In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure


# The data can be found in this address https://www.dubaipulse.gov.ae/data/dld-transactions/dld_transactions-open-api
df = pd.read_csv('Transactions.csv')

#Format date column
df.instance_date = pd.to_datetime(df.instance_date)
df.instance_date = pd.to_datetime(df.instance_date.dt.strftime('%Y-%m-%d'))

df = df.sort_values(by='instance_date', ascending=False)

#Remove the columns that aren't necessary for the analysis
df = df.drop(['transaction_id', 'procedure_id', 'trans_group_id', 'trans_group_ar',
        'procedure_name_ar', 
       'property_type_id', 'property_type_ar',
       'property_sub_type_id', 'property_sub_type_ar',
       'property_usage_ar', 
       'reg_type_id', 'reg_type_ar',  'area_id', 'area_name_ar',
       'building_name_ar', 'trans_group_en',
       'project_number', 'project_name_ar', 
       'master_project_ar', 'nearest_landmark_ar',
       'nearest_metro_ar',
       'nearest_mall_ar', 'nearest_mall_en', 'rooms_ar', 
       'rent_value', 'meter_rent_price', 'no_of_parties_role_1',
       'no_of_parties_role_2', 'no_of_parties_role_3'], axis=1, inplace=False)

In [ ]:
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
# Get rid of mortgage registrations and other non-relevant procedure types
df = df[(df.procedure_name_en=='Sell') | 
       (df.procedure_name_en=='Sell - Pre registration') | 
       (df.procedure_name_en=='Delayed Sell') | 
       (df.procedure_name_en=='Lease to Own Registration')]

#Remove Building and Land
df = df[(df.property_type_en =='Unit') |
        (df.property_type_en =='Villa')]

#Remove non-residential properties
df = df[df.property_usage_en == 'Residential']

#Remove white space in the below columns
df['area_name_en'] = df['area_name_en'].str.strip()
df['building_name_en'] = df['building_name_en'].str.strip()
df['project_name_en'] = df['project_name_en'].str.strip()
df['nearest_landmark_en'] = df['nearest_landmark_en'].str.strip()
df['nearest_metro_en'] = df['nearest_metro_en'].str.strip()

#Convert sqm to sqft
df.procedure_area = df.procedure_area*10.7639

In [ ]:
# Seperate DFs for units and villas and remove extreme floor areas
dfu = df[df.property_type_en=='Unit']
dfv = df[df.property_type_en=='Villa']

dfu = dfu[dfu.procedure_area<50000]
dfv = dfv[dfv.procedure_area<100000]

dfu = dfu[dfu.actual_worth>100000]
dfv = dfv[dfv.actual_worth>300000]

df = pd.concat([dfu,dfv], axis=0)
df = df[df.procedure_area > 370]

# After a quick check on the most expensive villas on bayut.com remove records with area larger than 100.000 sqft 
df = df[df.procedure_area<9290.304]

# Remove nans 
df.dropna(subset=['actual_worth','meter_sale_price' ], how='any', inplace=True)

# Remove below 100k properties
df = df[df.actual_worth>100000]

# A subset of 2023 data
df_2023 = df[df.instance_date > '2023-01-01']

# Remove non-residentials
df = df[df.property_usage_en == 'Residential']

In [ ]:
# A subset of the most expensive projects
pricey = df_2023.groupby(by='project_name_en').mean().sort_values(by='meter_sale_price', ascending=False).head(15)

# Another subset to look at the volume of sales
pricey_sold = pd.DataFrame(columns = ['project_name_en', 'unit sold'])

# Calculate the total number sold 
for i in pricey.index:
    pricey_sold.loc[len(pricey_sold)] = [i,sum(df_2023.project_name_en==i)]

# Remove the project that sold less than 3    
for proj in pricey_sold.project_name_en:
    if int(pricey_sold[pricey_sold.project_name_en==proj]['unit sold'])<3:
        pricey = pricey[pricey.index != proj]
# Keep the projects sold 3 and more        
pricey_sold = pricey_sold[pricey_sold['unit sold']>=3]

In [ ]:
"""  
date_formatted = []
for i in range(len(date)):
    date_formatted.append(date[i].strftime('%Y-%m-%d %H:%M:%S'))
  
date_formatted_villa = []
for i in range(len(a_villa)):
    date_formatted.append(a_villa[i].strftime('%Y-%m-%d %H:%M:%S'))
    
date_formatted_unit = []
for i in range(len(a_unit)):
    date_formatted.append(a_unit[i].strftime('%Y-%m-%d %H:%M:%S'))    
"""

In [ ]:
# Save various data to use in other visualization tools
"""
pricey.to_csv('highest_price_per_sqft.csv')
pricey_sold.to_csv('popularity_of_pricey.csv')
most_sold_proj.to_csv('most_sold_projects.csv')
most_sold_area.to_csv('hottest_areas.csv')
np.savetxt("downtown_prices_y.csv", y_.values, delimiter=",")
np.savetxt("downtown_prices_x.csv", x_.values, delimiter=",")
np.savetxt("downtown_date.csv", np.asarray(date_formatted), delimiter=",", fmt='%s')
np.savetxt("downtown_sales.csv", np.asarray(sales), delimiter=",")
b_unit.to_csv('unit_price_per_sqft.csv')
b_villa.to_csv('villa_price_per_sqft.csv')
"""

In [ ]:
figure(figsize=(20, 20), dpi=80)

plt.subplot(2, 1, 1)
plt.bar(pricey.index,pricey.meter_sale_price, width=0.25)
plt.xticks(rotation=18, ha='right', fontweight='bold')
plt.yticks(fontweight='bold')
plt.xlabel("PROJECT",fontweight='bold')
plt.ylabel("AED per squarefeet", fontweight='bold')
plt.text(5.5,115000,"Dubai's Highest Price per sqft Projects in 2023", ha='center', fontsize=30)


plt.subplot(2, 1, 2)
plt.bar(pricey_sold.project_name_en,pricey_sold['unit sold'],  width=0.25)
plt.xticks(rotation=18, ha='right', fontweight='bold')
plt.yticks(fontweight='bold')
plt.xlabel("PROJECT",fontweight='bold')
plt.ylabel("NUMBER OF UNITS SOLD", fontweight='bold')
plt.text(5.5,70,"Popularity of the projects ", ha='center', fontsize=30)

plt.savefig('test.png')

In [ ]:
# Subset the hottest projects and areas
most_sold_proj = df_2023.groupby(by='project_name_en').count().sort_values(by='meter_sale_price', ascending=False).head(15)
most_sold_area = df_2023.groupby(by='area_name_en').count().sort_values(by='meter_sale_price', ascending=False).head(15)


figure(figsize=(20, 10), dpi=80)
plt.bar(most_sold_proj.index, most_sold_proj.procedure_name_en,  width=0.25)
plt.xticks(rotation=18, ha='right', fontweight='bold')
plt.yticks(fontweight='bold')
plt.xlabel("Project",fontweight='bold', fontsize=15)
plt.ylabel("NUMBER OF UNITS SOLD", fontweight='bold', fontsize=15)
plt.text(6.5,750,"Hottest Projects of 2023 ", ha='center', fontsize=30)


In [ ]:
figure(figsize=(20, 10), dpi=80)
plt.bar(most_sold_area.index, most_sold_area.procedure_name_en,  width=0.25)
plt.xticks(rotation=18, ha='right', fontweight='bold')
plt.yticks(fontweight='bold')
plt.xlabel("AREA",fontweight='bold', fontsize=15)
plt.ylabel("NUMBER OF UNITS SOLD", fontweight='bold', fontsize=15)
plt.text(6.5,6000,"Hottest areas in 2023 ", ha='center', fontsize=30)
plt.text(7,2150,"*Downtown area", ha='center', fontsize=12)

In [ ]:
# Cheapest areas and projects for further analysis
affordable_area = df_2023.groupby(by='area_name_en').mean().sort_values(by='meter_sale_price', ascending=True).head(15)
affordable_proj = df_2023.groupby(by='project_name_en').mean().sort_values(by='meter_sale_price', ascending=True).head(15)

In [ ]:
# Subset the Downtown area and eliminate extreme cases 
dt = df[df.area_name_en=='Burj Khalifa']
dt = dt[dt.meter_sale_price<60000]

dt_volume = dt.groupby(by='instance_date').count()

In [ ]:
#Remove single data doint from 2017 that deviated the graph
dt_volume = dt_volume.iloc[1:,:]

date =[]
sales = []
for i in range(dt_volume.shape[0]//10):
    date.append(dt_volume.index[i*10])
    temp=0
    for j in range(10):
        temp = temp + dt_volume.iloc[i*10+j,0]
    sales.append(temp)

In [ ]:
x_ = dt.groupby(by='instance_date').mean().index
y_ = dt.groupby(by='instance_date').mean().meter_sale_price

dt_volume = dt.groupby(by='instance_date').count()
x_volume = dt_volume.groupby(by='instance_date').count().index
y_volume = dt_volume.groupby(by='instance_date').count().meter_sale_price


figure(figsize=(20, 20))

plt.subplot(2, 1, 1)
plt.scatter(x_,y_)

plt.xlabel("DATE",fontweight='bold')
plt.ylabel("AED per squarefeet in downtown", fontweight='bold')
plt.title("DOWNTOWN PRICES HAVE CAUGHT 2014 LEVELS", fontsize=25)


plt.subplot(2, 1, 2)
plt.scatter(date,sales)
plt.xlabel("DATE",fontweight='bold')
plt.ylabel("NUMBER OF TRANSACTIONS - 10 DAYS MOVING AVERAGE", fontweight='bold')
plt.title("VOLUME OF TRANSACTIONS ARE BACK TO 2014 LEVELS IN DOWNTOWN", fontsize=25)
#plt.text(0,0,"Market is as hot as 2014", ha='center', fontsize=1)

In [ ]:
# Most sold properties among the most expensive ones

pricey_area = df_2023.groupby(by='area_name_en').mean().sort_values(by='meter_sale_price', ascending=False).head(15)

pricey_sold = pd.DataFrame(columns = ['project_name_en', 'unit sold'])

for i in pricey.index:
    pricey_sold.loc[len(pricey_sold)] = [i,sum(df_2023.project_name_en==i)]
    
for proj in pricey_sold.project_name_en:
    if int(pricey_sold[pricey_sold.project_name_en==proj]['unit sold'])<3:
        pricey = pricey[pricey.index != proj]
        
pricey_sold = pricey_sold[pricey_sold['unit sold']>=3]

In [ ]:
df_2223 = df[df.instance_date>'2022-01-01']

In [ ]:
# Re-group units and villas to include only from 2022
unit = df[(df.property_type_en=='Unit')][df.instance_date>'2022-01-01']
villa = df[df.property_type_en=='Villa'][df.instance_date>'2022-01-01']

In [ ]:
# Date manipulation
import matplotlib.dates as dates
unit_num = dates.date2num(unit.groupby(by='instance_date').mean().index)
villa_num = dates.date2num(villa.groupby(by='instance_date').mean().index)

In [ ]:
figure(figsize=(20, 10))
import numpy as np
import matplotlib.dates as dates

a_unit=dates.date2num(unit.groupby(by='instance_date').mean().index)
b_unit= unit.groupby(by='instance_date').mean()['meter_sale_price']

a_villa=dates.date2num(villa.groupby(by='instance_date').mean().index)
b_villa= villa.groupby(by='instance_date').mean()['meter_sale_price']




z_unit = np.polyfit(a_unit, b_unit, 1)
p_unit = np.poly1d(z_unit)
plt.scatter(a_unit,b_unit)
plt.plot(a_unit,p_unit(a_unit),"bo--")


z_villa = np.polyfit(a_villa, b_villa, 1)
p_villa = np.poly1d(z_villa)
plt.scatter(a_villa,b_villa)
plt.plot(a_villa,p_villa(a_villa),color='orange', marker='o', linestyle='dashed')

plt.xticks([738200,738300,738400,738500,738600,738700,738800, 738840], ['Jan 2022','Mar 2022','June 2022','Sept 2022', 'Dec 2022', 'Mar 2023', 'June 2023', 'July 2023'])
plt.xlabel("DATE",fontweight='bold')
plt.ylabel("AED per squarefeet in Dubai", fontweight='bold')

plt.text(738500,24500,"Villas can be better investments than Units", ha='center', fontsize=25)

#plt.axvline(x=738150, ymin=8000, ymax=16100)

plt.axvline(x=738150, ymax=0.52, ymin=0.125, color='black')
plt.axvline(x=738870, ymax=0.65, ymin=0.32, color='black')
plt.legend(['UNITS', 'VILLAS'])



plt.savefig('villa_unit.png')